<a href="https://colab.research.google.com/github/TiagoIesbick/dashboard-etl/blob/main/dashboard_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import pathlib

### Expense

In [2]:
# Generating a dataframe with paid expenses data from 2002 to 2023

# Define the folder path
exp_folder_2002_2023 = pathlib.Path(r'/content/drive/MyDrive/Dashboard_data/Despesas/2002_2023')

# Read all excel files efficiently
df_list = [
    pd.read_excel(expense_file).dropna(axis=1, how='all')
    for expense_file in exp_folder_2002_2023.glob("*.xls")
]

# Concatenate all DataFrames
df_exp_2002_2023 = pd.concat(df_list, ignore_index=True)

# Rename and clean up columns
df_exp_2002_2023 = df_exp_2002_2023[['CPTPAG', 'RESUL_PAGO', 'UNIDORC', 'PROJATIV', 'RUBRICA', 'VINCORC']].rename(columns={
    'CPTPAG':'Comp.pagto.', 'RESUL_PAGO':'Result. pago', 'UNIDORC':'Unid. Orçam.', 'PROJATIV':'Proj/Ativ',
    'RUBRICA':'Rubrica', 'VINCORC':'Vinc. Orçam.'
})

# Filter out rows with nan values in some columns
df_exp_2002_2023 = df_exp_2002_2023.dropna(subset=['Comp.pagto.', 'Result. pago'], ignore_index=True)

# Process categorical column efficiently
df_exp_2002_2023['Elemento'] = df_exp_2002_2023['Rubrica'].astype(str).str[:6].astype(int)

In [3]:
# Generating a dataframe with paid expenses data from 2024 onwards

# Define the folder path
exp_folder_2024_onwards = pathlib.Path(r'/content/drive/MyDrive/Dashboard_data/Despesas/2024+')

# Read all CSV files efficiently
df_list = [
    pd.read_csv(expense_file).dropna(axis=1, how='all')
    for expense_file in exp_folder_2024_onwards.glob("*.csv")
]

# Concatenate all DataFrames
df_exp_2024_onwards = pd.concat(df_list, ignore_index=True)

# Convert financial columns efficiently
cols_to_convert = ['valorpago', 'restospagarnaoprocessadospagos', 'restospagarprocessadospagos']
df_exp_2024_onwards[cols_to_convert] = df_exp_2024_onwards[cols_to_convert].apply(
    lambda col: pd.to_numeric(col.str.replace('R\$ ', '', regex=True).str.replace(',', '.'), errors='coerce')
)

# Filter out rows with zero values in all columns to convert
df_exp_2024_onwards = df_exp_2024_onwards[
    df_exp_2024_onwards[cols_to_convert].ne(0).any(axis=1)
]

# Rename and clean up columns
df_exp_2024_onwards.rename(columns={
    'exercicio': 'YEAR', 'mes': 'MONTH', 'subacao': 'Proj/Ativ', 'unidadeorcamentaria': 'Unid. Orçam.'
}, inplace=True)

# Fix month values
df_exp_2024_onwards['MONTH'] = df_exp_2024_onwards['MONTH'].replace({0: 1, 13: 12}).astype(int)

# Convert date
df_exp_2024_onwards['Comp.pagto.'] = pd.to_datetime(df_exp_2024_onwards[['YEAR', 'MONTH']].assign(DAY=1))

# Process categorical columns efficiently
df_exp_2024_onwards['Elemento'] = df_exp_2024_onwards['elementocompleto'].str[:8].str.replace('.', '', regex=False).astype(int)
df_exp_2024_onwards['Rubrica'] = df_exp_2024_onwards['desdobramentocompleto'].str[:12].str.replace('.', '', regex=False).astype(int)

# Sorting once before processing
df_exp_2024_onwards.sort_values('Comp.pagto.', inplace=True, ignore_index=True)

# Subtract the last nonzero appearance for target columns
cols = ['Proj/Ativ', 'Rubrica', 'fonterecursos', 'YEAR']
target_cols = ['restospagarnaoprocessadospagos', 'restospagarprocessadospagos']
for target_col in target_cols:
    df_exp_2024_onwards[target_col + '_prev'] = (
        df_exp_2024_onwards.where(df_exp_2024_onwards[target_col] != 0).groupby(cols)[target_col].shift(1).fillna(0)
    )
    df_exp_2024_onwards[target_col] -= df_exp_2024_onwards[target_col + '_prev']

# Compute final results
df_exp_2024_onwards['Result. pago'] = df_exp_2024_onwards[cols_to_convert].sum(axis=1)

# Final filter and reordering columns
df_exp_2024_onwards = df_exp_2024_onwards[df_exp_2024_onwards['Result. pago'] != 0].reset_index(drop=True)
df_exp_2024_onwards['Vinc. Orçam.'] = df_exp_2024_onwards['defonterecursos'].str[:4].astype(int)

df_exp_2024_onwards = df_exp_2024_onwards[
    ['Comp.pagto.', 'Result. pago', 'Unid. Orçam.', 'Proj/Ativ', 'Elemento', 'Rubrica', 'Vinc. Orçam.',
     'deelemento', 'dedesdobramento', 'desubacao', 'deunidadeorcamentária', 'defonterecursos']
]

In [4]:
# concatenating the expense dataframes
relevant_columns = ['Comp.pagto.', 'Result. pago', 'Unid. Orçam.', 'Proj/Ativ', 'Elemento', 'Rubrica', 'Vinc. Orçam.']
df_exp = pd.concat([df_exp_2002_2023, df_exp_2024_onwards[relevant_columns]], ignore_index=True)

**Revenue**

In [5]:
# Generating a dataframe with revenue collected from 2004 to 2017

revenue_file_2004_2017 = r'/content/drive/MyDrive/Dashboard_data/Receitas/2004_2017/2004_2017_Cubo_Antigo.xlsx'

df_rev_2004_2017 = pd.read_excel(revenue_file_2004_2017)
df_rev_2004_2017.drop(0, inplace=True)
df_rev_2004_2017.drop('Valor Orçado', axis=1, inplace=True)
df_rev_2004_2017.rename(columns={'Exercício': 'YEAR',
                                 'Mês': 'MONTH',
                                 'Rótulos de Linha': 'orgao',
                                 'Vínculo Cod': 'vinculo',
                                 'N6 Subalinea': 'desdobramento6',
                                 'Valor Arrecadado': 'valor_arrecadado'}, inplace=True)
df_rev_2004_2017['Data'] = pd.to_datetime(df_rev_2004_2017[['YEAR', 'MONTH']].assign(DAY=1))
df_rev_2004_2017[['YEAR', 'orgao', 'MONTH', 'vinculo']] = df_rev_2004_2017[['YEAR', 'orgao', 'MONTH', 'vinculo']].astype('int64')
df_rev_2004_2017 = df_rev_2004_2017.loc[df_rev_2004_2017['valor_arrecadado'] != 0].reset_index(drop=True)
df_rev_2004_2017.sort_values('Data', inplace=True, ignore_index=True)

df_rev_from_to_2004_2017 = pd.read_excel(r'/content/drive/MyDrive/Dashboard_data/utils/rev_from_to-2004_2017.xlsx')
df_rev_from_to_2004_2017['rubrica'] = df_rev_from_to_2004_2017['rubricacompleta'].str.extract(r'^(\d+)')
df_rev_from_to_2004_2017["nome_rubrica"] = df_rev_from_to_2004_2017["rubricacompleta"].str.extract(r"^\d+\s*-\s*(.*)")

df_rev_2004_2017 = df_rev_2004_2017.merge(
    df_rev_from_to_2004_2017[['vinculo', 'desdobramento6', 'rubrica', 'nome_rubrica']],
    on=['vinculo', 'desdobramento6'], how='left'
)

In [6]:
# Generating a dataframe with revenue collected from 2018 to 2023

rev_folder_2018_2023 = pathlib.Path(r'/content/drive/MyDrive/Dashboard_data/Receitas/2018_2023')

df_list = [
    pd.read_excel(revenue_file).dropna(axis=1, how='all')
    for revenue_file in rev_folder_2018_2023.glob("*.xlsx")
]

df_rev_2018_2023 = pd.concat(df_list, ignore_index=True)

df_rev_2018_2023.drop(columns=[
    'valor_cancelado', 'valor_orcado', 'valor_lancado', 'valor_meta', 'informacao_complementar',
    'nome_orgao', 'orgao', 'nome_orgao.1', 'digito', 'categoria', 'especie', 'desdobramento1',
    'desdobramento2', 'desdobramento3', 'desdobramento4', 'desdobramento5'
    ], inplace=True)

df_rev_2018_2023 = df_rev_2018_2023.loc[(df_rev_2018_2023['orgao_raiz'] == 7000) & (df_rev_2018_2023['valor_arrecadado'] != 0)].reset_index(drop=True)
df_rev_2018_2023.rename(columns={'ano': 'YEAR', 'mes': 'MONTH'}, inplace=True)
df_rev_2018_2023['Data'] = pd.to_datetime(df_rev_2018_2023[['YEAR', 'MONTH']].assign(DAY=1))
df_rev_2018_2023.sort_values('Data', inplace=True, ignore_index=True)

df_rev_from_to_2018_2023 = pd.read_excel(r'/content/drive/MyDrive/Dashboard_data/utils/rev_from_to-2018_2023.xlsx')
df_rev_from_to_2018_2023['rubrica_new'] = df_rev_from_to_2018_2023['rubricacompleta'].str.extract(r'^(\d+)')
df_rev_from_to_2018_2023["nome_rubrica"] = df_rev_from_to_2018_2023["rubricacompleta"].str.extract(r"^\d+\s*-\s*(.*)")

df_rev_2018_2023 = df_rev_2018_2023.merge(
    df_rev_from_to_2018_2023.drop('rubricacompleta', axis=1), on='rubrica', how='left'
    ).drop('rubrica', axis=1).rename(columns={'rubrica_new': 'rubrica'})

In [7]:
# Generating a dataframe with revenue collected from 2024 onwards

rev_folder_2024_onwards = pathlib.Path(r'/content/drive/MyDrive/Dashboard_data/Receitas/2024+')

df_list = [
    pd.read_csv(revenue_file).dropna(axis=1, how='all')
    for revenue_file in rev_folder_2024_onwards.glob("*.csv")
]

df_rev_2024_onwards = pd.concat(df_list, ignore_index=True)

df_rev_2024_onwards.drop(columns=[
    'orgao', 'unidadegestora', 'gestao', 'unidadeorcamentaria', 'categoriaeconomica',
    'especie', 'd1', 'dd2', 'd3', 'desdobramentosigef', 'codificacaotce', 'valorarrecadadobruto',
    'valoratualizadoliquido', 'valordeducao', 'valordeducaoatualizado', 'valordeducaoorcado',
    'valororcadoliquido', 'valorreceitaatualizado', 'valorreceitaorcado'
    ], inplace=True)

df_rev_2024_onwards.rename(columns={'exercicio': 'YEAR', 'mes': 'MONTH'}, inplace=True)
df_rev_2024_onwards['Data'] = pd.to_datetime(df_rev_2024_onwards[['YEAR', 'MONTH']].assign(DAY=1))
df_rev_2024_onwards.sort_values('Data', inplace=True, ignore_index=True)

df_rev_2024_onwards['rubrica'] = df_rev_2024_onwards['rubricacompleta'].str.extract(r'^(\d+)')
df_rev_2024_onwards["nome_rubrica"] = df_rev_2024_onwards["rubricacompleta"].str.extract(r"^\d+\s*-\s*(.*)")

In [8]:
# Generating normalization dataframe of origin, type and rubric

df_otr = df_rev_2024_onwards[['origem', 'tipo', 'rubrica', 'nome_rubrica']].drop_duplicates(keep='last').rename(
    columns={'origem': 'o', 'tipo': 't', 'rubrica': 'r_int', 'nome_rubrica': 'nome_r'}
)

for column in ['o', 't']:
  df_otr[column + '_int'] = df_otr[column].str.extract(r'^(\d+)')
  df_otr['nome_' + column] = df_otr[column].str.extract(r"^\d+\s*-\s*(.*)")

df_tipo = pd.read_excel(r'/content/drive/MyDrive/Dashboard_data/utils/df_tipo.xlsx', dtype=str)
df_tipo['tipo_int'] = df_tipo['tipo'].str.extract(r'^(\d+)')
df_tipo["nome_tipo"] = df_tipo['tipo'].str.extract(r"^\d+\s*-\s*(.*)")
df_tipo = df_tipo.merge(
    df_otr[['t_int', 'nome_t']].drop_duplicates(subset='t_int', keep='last'),
    left_on='tipo_int', right_on='t_int', how='left'
    )
df_tipo.loc[df_tipo['nome_t'].isna(), ['t_int', 'nome_t']] = df_tipo.loc[df_tipo['nome_t'].isna(), ['tipo_int', 'nome_tipo']].values
df_tipo = df_tipo[['rubrica', 't_int', 'nome_t']].rename(columns={'rubrica': 'r_int'})

df_otr = pd.concat([df_otr, df_tipo], ignore_index=True)

In [9]:
# Filter out rows with nan or zero values in relevant column

df_rev_2024_onwards = df_rev_2024_onwards.dropna(subset='valorrealizadoliquido', ignore_index=True)
df_rev_2024_onwards = df_rev_2024_onwards.loc[df_rev_2024_onwards['valorrealizadoliquido'] != 0].reset_index(drop=True)
df_rev_2024_onwards['vinculo'] = df_rev_2024_onwards['fonterecurso'].str.extract(r"^\d+\s*-\s*(\d+)").astype(int)
df_rev_2024_onwards.rename(columns={'valorrealizadoliquido': 'valor_arrecadado'}, inplace=True)

In [10]:
# Concatenating the revenue dataframes and normalizing origin, type and rubric

df_rev = pd.concat([df_rev_2004_2017, df_rev_2018_2023, df_rev_2024_onwards], ignore_index=True)
df_rev = df_rev.merge(df_otr, left_on='rubrica', right_on='r_int', how='left')
df_rev.loc[~df_rev['nome_r'].isna(), 'origem'] = df_rev['nome_r']
df_rev.loc[~df_rev['nome_t'].isna(), 'tipo'] = df_rev['nome_t']
df_rev.loc[~df_rev['nome_o'].isna(), 'origem'] = df_rev['nome_o']
df_rev.loc[df_rev['nome_o'].isna(), 'origem'] = df_rev['rubrica'].str[:2].map(
    df_otr[['o_int', 'nome_o']].drop_duplicates(keep='last').set_index('o_int')['nome_o']
    )
df_rev.loc[
    (df_rev['origem'].isna()) &
    (df_rev['rubrica'].str[:2] == '17'), 'origem'] = 'Transferências Correntes'

### Files

In [11]:
# Reading auxiliary tables

file_path = r'/content/drive/MyDrive/Dashboard_data/utils/auxiliary_tables.xlsx'

sheets_info = {
    'n_uni': {'skiprows': 2, 'cols': ['uni']},
    'n_proj': {'skiprows': 3, 'cols': ['proj', 'nome_proj']},
    'n_elem': {'skiprows': 3, 'cols': ['elem', 'nome_elem']},
    'n_vinc': {'skiprows': 3, 'cols': ['vinc', 'nome_vinc']},
    'n_rub': {'skiprows': 3, 'cols': ['rub', 'nome_rub']}
}

dfs = {
    sheet: pd.read_excel(file_path, sheet_name=sheet, skiprows=info['skiprows'], header=None, names=info['cols'])
    for sheet, info in sheets_info.items()
}

for sheet, df in dfs.items():
  if 'uni' in df.columns:
      df['nome_uni'] = df['uni'].str[5:]
      df['uni_int'] = df['uni'].str[:4].astype(int)
  if 'proj' in df.columns:
      df['proj_int'] = df['proj'].str[:4].astype(int)
  if 'elem' in df.columns:
      df['elem_int'] = df['elem'].str[:6].astype(int)
  if 'rub' in df.columns:
      df['rub_int'] = df['rub'].str[:12].astype('int64')
  if 'vinc' in df.columns:
      df['vinc_int'] = df['vinc'].str[:4].astype(int)

df_uni, df_proj, df_elem, df_vinc, df_rub = dfs.values()

In [12]:
# Updating the elements dataframe

df_new_elem = df_exp_2024_onwards[['Elemento', 'deelemento']].drop_duplicates(keep='last').rename(columns={
    'Elemento': 'elem_int',
    'deelemento': 'nome_elem'
})
df_elem = pd.concat([df_elem, df_new_elem], ignore_index=True).drop_duplicates(subset='elem_int', keep='last').reset_index(drop=True)
df_elem['elem'] = df_elem['elem_int'].astype(str) + ' - ' + df_elem['nome_elem'].str.title()

In [13]:
# Updating the rubric dataframe

df_new_rub = df_exp_2024_onwards[['Rubrica', 'dedesdobramento']].drop_duplicates(keep='last').rename(columns={
    'Rubrica': 'rub_int',
    'dedesdobramento': 'nome_rub'
})
df_rub = pd.concat([df_rub, df_new_rub], ignore_index=True).drop_duplicates(subset='rub_int', keep='last').reset_index(drop=True)
df_rub['rub'] = df_rub['rub_int'].astype(str) + ' - ' + df_rub['nome_rub'].str.title()

In [14]:
# Updating the project dataframe

df_new_proj = df_exp_2024_onwards[['Proj/Ativ', 'desubacao']].drop_duplicates(keep='last').rename(columns={
    'Proj/Ativ': 'proj_int',
    'desubacao': 'nome_proj'
})
df_proj = pd.concat([df_proj, df_new_proj], ignore_index=True).drop_duplicates(subset='proj_int', keep='last').reset_index(drop=True)
df_proj['proj'] = df_proj['proj_int'].astype(str) + ' - ' + df_proj['nome_proj'].str.title()

In [15]:
# Updating the budget units dataframe

df_new_uni = df_exp_2024_onwards[['Unid. Orçam.', 'deunidadeorcamentária']].drop_duplicates(keep='last').rename(columns={
    'Unid. Orçam.': 'uni_int',
    'deunidadeorcamentária': 'nome_uni'
})
df_uni = pd.concat([df_uni, df_new_uni], ignore_index=True).drop_duplicates(subset='uni_int', keep='last').reset_index(drop=True)
df_uni['uni'] = df_uni['uni_int'].astype(str) + ' - ' + df_uni['nome_uni'].str.title()

In [16]:
# Updating the entail dataframe

df_new_vinc_exp = df_exp_2024_onwards[['Vinc. Orçam.', 'defonterecursos']].drop_duplicates(keep='last').rename(columns={
    'Vinc. Orçam.': 'vinc_int', 'defonterecursos': 'nome_vinc'
})
df_new_vinc_exp['nome_vinc'] = df_new_vinc_exp['nome_vinc'].str.split('-').str.get(1).str.strip()

df_new_vinc_rev = df_rev_2024_onwards[['fonterecurso', 'vinculo']].drop_duplicates(keep='last').rename(columns={
    'vinculo': 'vinc_int', 'fonterecurso': 'nome_vinc'
})
df_new_vinc_rev['nome_vinc'] = df_new_vinc_rev['nome_vinc'].str.extract(r"^\d+\s*-\s*\d+\s*-\s*(.*)")

df_vinc = pd.concat([df_vinc, df_new_vinc_exp, df_new_vinc_rev], ignore_index=True).drop_duplicates(subset='vinc_int', keep='last').reset_index(drop=True)

common_values = set(df_exp['Vinc. Orçam.']) | set(df_rev['vinculo'])
df_vinc = df_vinc[df_vinc['vinc_int'].isin(common_values)]

df_vinc['vinc'] = df_vinc['vinc_int'].astype(str) + ' - ' + df_vinc['nome_vinc'].str.title()

In [17]:
# Saving the files

final_data_folder = pathlib.Path(r'/content/drive/MyDrive/Dashboard_data/final_data')

df_uni.sort_values('uni_int').to_csv(final_data_folder / 'df_uni.csv', encoding='utf-8-sig', sep=';', index=False)
df_proj.sort_values('proj_int').to_csv(final_data_folder / 'df_proj.csv', encoding='utf-8-sig', sep=';', index=False)
df_elem.sort_values('elem_int').to_csv(final_data_folder / 'df_elem.csv', encoding='utf-8-sig', sep=';', index=False)
df_rub.sort_values('rub_int').to_csv(final_data_folder / 'df_rub.csv', encoding='utf-8-sig', sep=';', index=False)
df_vinc.sort_values('vinc_int').to_csv(final_data_folder / 'df_vinc.csv', encoding='utf-8-sig', sep=';', index=False)
df_exp.sort_values('Comp.pagto.').to_csv(final_data_folder / 'df_exp.csv', encoding='utf-8-sig', sep=';', index=False)
df_rev[['Data', 'vinculo', 'origem', 'tipo', 'nome_rubrica', 'valor_arrecadado']].sort_values('Data').to_csv(
    final_data_folder / 'df_rev.csv', encoding='utf-8-sig', sep=';', index=False
)